# Step Four: Analytical ETL

Use SparkSQL and Python to build an ETL job that calculates the following results for a given day:<br>
&emsp;&emsp;-Latest trade price as of each quote. <br>
&emsp;&emsp;-Latest 3 hour moving average trade price, before the quote. <br>
&emsp;&emsp;-The bid/ask price movement from the previous day’s closing price.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window


## 4.1 Read Parquet Files From Azure Blob Storage Partition

In [0]:
df =spark.read.parquet("/trade/trade_dt={}".format("2020-08-06"))

## 4.2 Create Trade Staging Table
### 4.2.1 Use Spark To Read The Trade Table With Date Partition “2020-07-29”

In [0]:
df = df.select("stock_symbol","stock_exchange","latest_trade", "event_seq_nb", "trade_pr").where(df.trade_dt == "2020-08-06")

### 4.2.2 Create A Spark Temporary View

In [0]:
df.createOrReplaceTempView("tmp_trade_moving_avg")

In [0]:
spark.sql("SELECT stock_symbol,stock_exchange,latest_trade,event_seq_nb,trade_pr FROM tmp_trade_moving_avg").show(truncate=False)

+------------+--------------+-----------------------+------------+---------+
stock_symbol|stock_exchange|latest_trade |event_seq_nb|trade_pr |
+------------+--------------+-----------------------+------------+---------+
SYMA |NYSE |2020-08-06 10:49:37.345|10 |74.493774|
SYMA |NASDAQ |2020-08-06 12:00:29.595|20 |77.0967 |
SYMA |NYSE |2020-08-06 13:11:57.308|30 |76.899635|
SYMA |NYSE |2020-08-06 14:27:13.014|40 |77.11552 |
SYMA |NYSE |2020-08-06 15:39:08.521|50 |76.37442 |
SYMA |NYSE |2020-08-06 16:58:58.633|60 |78.32364 |
SYMA |NYSE |2020-08-06 18:14:28.899|70 |78.23471 |
SYMA |NYSE |2020-08-06 19:28:22.629|80 |76.528984|
SYMA |NYSE |2020-08-06 20:49:10.946|90 |75.71484 |
SYMA |NYSE |2020-08-06 22:00:18.406|100 |76.30847 |
SYMB |NASDAQ |2020-08-06 10:47:40.723|10 |32.27114 |
SYMB |NYSE |2020-08-06 12:01:40.839|20 |32.92886 |
SYMB |NYSE |2020-08-06 13:13:27.041|30 |33.686092|
SYMB |NYSE |2020-08-06 14:29:42.34 |40 |35.07168 |
SYMB |NYSE |2020-08-06 15:46:21.701|50 |34.834843|
SYMB |NYSE |2020-08-06 16:57:15.217|60 |33.31802 |
SYMB |NYSE |2020-08-06 18:06:29.531|70 |33.565834|
SYMB |NYSE |2020-08-06 19:21:50.111|80 |33.10892 |
SYMB |NYSE |2020-08-06 20:36:05.757|90 |32.643288|
SYMB |NYSE |2020-08-06 21:46:00.523|100 |35.924744|
+------------+--------------+-----------------------+------------+---------+
only showing top 20 rows

### 4.2.3 Calculate The 180*-min Moving Average Using The Spark Temp View 
Partition by symbol and order by time. The window should contain all records within 30-min of the corresponding row.<br>
*Increased from 30mins b/c the given data only has trades every ~ 90 mins, so 30min mov avg is identical to trade_pr.

In [0]:
mov_avg_df = spark.sql("SELECT stock_symbol,stock_exchange, latest_trade, event_seq_nb, trade_pr, \
    AVG(trade_pr) OVER (PARTITION BY stock_symbol ORDER BY latest_trade RANGE BETWEEN INTERVAL '180' MINUTES PRECEDING AND CURRENT ROW) AS mov_avg_pr \
    FROM tmp_trade_moving_avg")

In [0]:
mov_avg_df.show(3,truncate=False)

+------------+--------------+-----------------------+------------+---------+-----------------+
stock_symbol|stock_exchange|latest_trade |event_seq_nb|trade_pr |mov_avg_pr |
+------------+--------------+-----------------------+------------+---------+-----------------+
SYMA |NYSE |2020-08-06 10:49:37.345|10 |74.493774|74.4937744140625 |
SYMA |NASDAQ |2020-08-06 12:00:29.595|20 |77.0967 |75.79523849487305|
SYMA |NYSE |2020-08-06 13:11:57.308|30 |76.899635|76.16337076822917|
+------------+--------------+-----------------------+------------+---------+-----------------+
only showing top 3 rows

### 4.2.4 Save The Temporary View Into Hive Table For Staging

In [0]:
mov_avg_df.write.mode('overwrite').saveAsTable("temp_trade_moving_avg")

## 4.3 Create Staging Table For The Prior Day’s Last Trade
You’ll need another staging table in order to make the last trade price available to the quote
records of corresponding symbols. Note that this table has only 1 price per trade/exchange, so
the record size will be small enough.

### 4.3.1 Get The Previous Date Value

In [0]:
import datetime
date = datetime.datetime.strptime('2020-08-06', '%Y-%m-%d')
prev_date_str = (date - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
print('Assuming current date of 2020-08-06, prev_date_str = ' + prev_date_str)

Assuming current date of 2020-08-06, prev_date_str = 2020-08-05

### 4.3.2 Use Spark To Read The Trade Table With Date Partition “2020-07-28”

In [0]:
df =spark.read.parquet("/trade/trade_dt={}".format(prev_date_str))
df = df.select("stock_symbol","stock_exchange","latest_trade", "event_seq_nb", "trade_pr").where(df.trade_dt == prev_date_str)
df.show(truncate=False)

+------------+--------------+-----------------------+------------+---------+
stock_symbol|stock_exchange|latest_trade |event_seq_nb|trade_pr |
+------------+--------------+-----------------------+------------+---------+
SYMA |NASDAQ |2020-08-05 10:38:50.046|10 |77.7757 |
SYMA |NASDAQ |2020-08-05 11:58:33.106|20 |75.715225|
SYMA |NASDAQ |2020-08-05 13:09:24.38 |30 |75.87926 |
SYMA |NYSE |2020-08-05 14:24:34.33 |40 |78.430954|
SYMA |NASDAQ |2020-08-05 15:33:58.825|50 |75.72602 |
SYMA |NASDAQ |2020-08-05 16:46:43.764|60 |77.479485|
SYMA |NASDAQ |2020-08-05 17:58:10.324|70 |74.91233 |
SYMA |NASDAQ |2020-08-05 19:13:26.116|80 |77.69288 |
SYMA |NASDAQ |2020-08-05 20:26:46.538|90 |76.15561 |
SYMA |NASDAQ |2020-08-05 21:40:23.197|100 |77.24676 |
SYMB |NYSE |2020-08-05 10:43:54.198|10 |34.978676|
SYMB |NYSE |2020-08-05 12:02:40.506|20 |33.17828 |
SYMB |NYSE |2020-08-05 13:10:45.361|30 |34.183914|
SYMB |NYSE |2020-08-05 14:26:20.473|40 |36.511795|
SYMB |NYSE |2020-08-05 15:31:43.643|50 |36.07549 |
SYMB |NYSE |2020-08-05 16:48:29.503|60 |35.762054|
SYMB |NYSE |2020-08-05 18:02:11.213|70 |32.681087|
SYMB |NYSE |2020-08-05 19:08:12.441|80 |34.27408 |
SYMB |NYSE |2020-08-05 20:17:22.625|90 |34.2493 |
SYMB |NYSE |2020-08-05 21:27:28.314|100 |33.956287|
+------------+--------------+-----------------------+------------+---------+
only showing top 20 rows

### 4.3.3 Create Spark Temporary View

In [0]:
df.createOrReplaceTempView("tmp_last_trade")

### 4.3.4 Calculate Last Trade Price Using The Spark Temp View

In [0]:
""" 
THIS QUERY TEMPLATE IN THE INSTRUCTIONS IS NOT CLEAR - WHY DID I JUST MAKE TMP_LAST_TRADE IF I'M RE-USING TMP_TRADE_MOVING_AVG FROM BEFORE? 

last_pr_df = spark.sql("select symbol, exchange, last_pr from (select
symbol, exchange, event_tm, event_seq_nb, trade_pr,
# [logic to derive last 30 min moving average price] AS last_pr
FROM tmp_trade_moving_avg) a
")

"""

In [0]:
# This logic follows the lead from the query above (from instructions)
last_pr_df = spark.sql("SELECT stock_symbol, stock_exchange, latest_trade, event_seq_nb,trade_pr, last_mov_avg_pr FROM \
(SELECT stock_symbol, stock_exchange, latest_trade,  event_seq_nb, trade_pr,LAST_VALUE(mov_avg_pr) \
 OVER (PARTITION BY stock_symbol ORDER BY latest_trade RANGE BETWEEN INTERVAL '180' MINUTES PRECEDING AND CURRENT ROW) \
 AS last_mov_avg_pr FROM temp_trade_moving_avg) as tmp2b")
last_pr_df.show(3,truncate=False)

# The following, MUCH simpler query also works because:
# THE MOVING AVG PRICE AS OF ANY GIVEN TIMESTAMP IS INHERENTLY THE LATEST MOVING AVG PRICE AS OF THAT TIMESTAMP.  THE PREMISE MAKES NO SENSE.
last_pr_df2 = spark.sql("SELECT stock_symbol, stock_exchange, latest_trade,  event_seq_nb, trade_pr, mov_avg_pr as last_mov_avg_pr FROM temp_trade_moving_avg")
last_pr_df2.show(3,truncate=False)

+------------+--------------+-----------------------+------------+---------+-----------------+
stock_symbol|stock_exchange|latest_trade |event_seq_nb|trade_pr |last_mov_avg_pr |
+------------+--------------+-----------------------+------------+---------+-----------------+
SYMA |NYSE |2020-08-06 10:49:37.345|10 |74.493774|74.4937744140625 |
SYMA |NASDAQ |2020-08-06 12:00:29.595|20 |77.0967 |75.79523849487305|
SYMA |NYSE |2020-08-06 13:11:57.308|30 |76.899635|76.16337076822917|
+------------+--------------+-----------------------+------------+---------+-----------------+
only showing top 3 rows

+------------+--------------+-----------------------+------------+---------+-----------------+
stock_symbol|stock_exchange|latest_trade |event_seq_nb|trade_pr |last_mov_avg_pr |
+------------+--------------+-----------------------+------------+---------+-----------------+
SYMA |NYSE |2020-08-06 10:49:37.345|10 |74.493774|74.4937744140625 |
SYMA |NASDAQ |2020-08-06 12:00:29.595|20 |77.0967 |75.79523849487305|
SYMA |NYSE |2020-08-06 13:11:57.308|30 |76.899635|76.16337076822917|
+------------+--------------+-----------------------+------------+---------+-----------------+
only showing top 3 rows

In [0]:
prev_last_pr_df = spark.sql("SELECT stock_symbol, stock_exchange, latest_trade,  event_seq_nb, trade_pr, \
LAST_VALUE(mov_avg_pr) OVER (PARTITION BY stock_symbol ORDER BY latest_trade RANGE BETWEEN INTERVAL '180' MINUTES PRECEDING AND CURRENT ROW) AS last_mov_avg_pr \
                            FROM (SELECT stock_symbol,stock_exchange, latest_trade, event_seq_nb, trade_pr, \
                                  AVG(trade_pr) OVER (PARTITION BY stock_symbol ORDER BY latest_trade RANGE BETWEEN INTERVAL '180' MINUTES PRECEDING AND CURRENT ROW) AS mov_avg_pr \
                                  FROM tmp_last_trade) as prev_tmp_trade_mov_avg")
prev_last_pr_df.show(3,truncate=False)
# Again, the following, MUCH simpler query also works because:
# THE MOVING AVG PRICE AS OF ANY GIVEN TIMESTAMP IS INHERENTLY THE LATEST MOVING AVG PRICE AS OF THAT TIMESTAMP.  THE PREMISE MAKES NO SENSE.
# Either the instructions are intentionally ambiguous, or I made a wrong turn awhile ago
prev_last_pr_df2 = spark.sql("SELECT stock_symbol, stock_exchange, latest_trade, event_seq_nb, trade_pr, mov_avg_pr AS last_mov_avg_pr \
                            FROM (SELECT stock_symbol,stock_exchange, latest_trade, event_seq_nb, trade_pr, \
                                  AVG(trade_pr) OVER (PARTITION BY stock_symbol ORDER BY latest_trade RANGE BETWEEN INTERVAL '180' MINUTES PRECEDING AND CURRENT ROW) AS mov_avg_pr \
                                  FROM tmp_last_trade) as prev_tmp_trade_mov_avg")
prev_last_pr_df.show(3,truncate=False)

+------------+--------------+-----------------------+------------+---------+-----------------+
stock_symbol|stock_exchange|latest_trade |event_seq_nb|trade_pr |last_mov_avg_pr |
+------------+--------------+-----------------------+------------+---------+-----------------+
SYMA |NASDAQ |2020-08-05 10:38:50.046|10 |77.7757 |77.77570343017578|
SYMA |NASDAQ |2020-08-05 11:58:33.106|20 |75.715225|76.74546432495117|
SYMA |NASDAQ |2020-08-05 13:09:24.38 |30 |75.87926 |76.45672861735027|
+------------+--------------+-----------------------+------------+---------+-----------------+
only showing top 3 rows

+------------+--------------+-----------------------+------------+---------+-----------------+
stock_symbol|stock_exchange|latest_trade |event_seq_nb|trade_pr |last_mov_avg_pr |
+------------+--------------+-----------------------+------------+---------+-----------------+
SYMA |NASDAQ |2020-08-05 10:38:50.046|10 |77.7757 |77.77570343017578|
SYMA |NASDAQ |2020-08-05 11:58:33.106|20 |75.715225|76.74546432495117|
SYMA |NASDAQ |2020-08-05 13:09:24.38 |30 |75.87926 |76.45672861735027|
+------------+--------------+-----------------------+------------+---------+-----------------+
only showing top 3 rows

### 4.3.4 Save The Temporary View Into Hive Table For Staging

In [0]:
# Instructions show: 
# mov_avg_df.write.saveAsTable("temp_last_trade")
# ^ is this a typo? meant to be below? ugh.

In [0]:
last_pr_df.write.mode('overwrite').saveAsTable("temp_last_trade")

In [0]:
prev_last_pr_df.write.mode('overwrite').saveAsTable("prev_temp_last_trade")

## 4.4 Populate The Latest Trade and Latest Moving Average Trade Price To The Quote Records
Now that you’ve produced both staging tables, join them with the main table “quotes” to populate trade related information.

In [0]:
# the tables are the exact same, with the exception of the latter having been created with additional column selected: event_seq_nb
# what was the point of this?
print('first rows of current day, in ascending order:')
spark.sql("select * from temp_last_trade where stock_symbol = 'SYMC'").show(3,truncate=False)
prev_last_pr_df_ind = prev_last_pr_df.withColumn("index", monotonically_increasing_id())
print('last rows of previous day, in descending order:')
prev_last_pr_df_ind.orderBy(desc("index")).drop("index").show(5,truncate=False)
spark.sql("select * from temp_last_trade").show(3,truncate=False)
print('now an identical table:')
spark.sql("select * from temp_trade_moving_avg").show(3,truncate=False)

first rows of current day, in ascending order:
+------------+--------------+-----------------------+------------+---------+------------------+
stock_symbol|stock_exchange|latest_trade |event_seq_nb|trade_pr |last_mov_avg_pr |
+------------+--------------+-----------------------+------------+---------+------------------+
SYMC |NASDAQ |2020-08-06 10:44:32.704|10 |158.81964|158.81964111328125|
SYMC |NASDAQ |2020-08-06 11:54:16.544|20 |160.77866|159.7991485595703 |
SYMC |NASDAQ |2020-08-06 13:07:41.026|30 |160.12585|159.90805053710938|
+------------+--------------+-----------------------+------------+---------+------------------+
only showing top 3 rows

last rows of previous day, in descending order:
+------------+--------------+-----------------------+------------+---------+------------------+
stock_symbol|stock_exchange|latest_trade |event_seq_nb|trade_pr |last_mov_avg_pr |
+------------+--------------+-----------------------+------------+---------+------------------+
SYMC |NYSE |2020-08-05 21:52:38.607|100 |160.61949|160.4873504638672 |
SYMC |NYSE |2020-08-05 20:48:19.795|90 |159.71748|159.67370096842447|
SYMC |NYSE |2020-08-05 19:37:55.659|80 |161.12508|159.91700744628906|
SYMC |NYSE |2020-08-05 18:20:43.709|70 |158.17854|158.5649210611979 |
SYMC |NASDAQ |2020-08-05 17:02:58.71 |60 |160.4474 |158.7165781656901 |
+------------+--------------+-----------------------+------------+---------+------------------+
only showing top 5 rows

+------------+--------------+-----------------------+------------+---------+-----------------+
stock_symbol|stock_exchange|latest_trade |event_seq_nb|trade_pr |last_mov_avg_pr |
+------------+--------------+-----------------------+------------+---------+-----------------+
SYMA |NYSE |2020-08-06 10:49:37.345|10 |74.493774|74.4937744140625 |
SYMA |NASDAQ |2020-08-06 12:00:29.595|20 |77.0967 |75.79523849487305|
SYMA |NYSE |2020-08-06 13:11:57.308|30 |76.899635|76.16337076822917|
+------------+--------------+-----------------------+------------+---------+-----------------+
only showing top 3 rows

now an identical table, with another trade column that the instructions say is only relevant for quotes:
+------------+--------------+-----------------------+------------+---------+-----------------+
stock_symbol|stock_exchange|latest_trade |event_seq_nb|trade_pr |mov_avg_pr |
+------------+--------------+-----------------------+------------+---------+-----------------+
SYMA |NYSE |2020-08-06 10:49:37.345|10 |74.493774|74.4937744140625 |
SYMA |NASDAQ |2020-08-06 12:00:29.595|20 |77.0967 |75.79523849487305|
SYMA |NYSE |2020-08-06 13:11:57.308|30 |76.899635|76.16337076822917|
+------------+--------------+-----------------------+------------+---------+-----------------+
only showing top 3 rows

### 4.4.1 Join With Table temp_trade_moving_avg
You need to join “quotes” and “temp_trade_moving_avg” to populate trade_pr and mov_avg_pr into quotes. However, you cannot use equality join in this case; trade events don’t happen at the same quote time. You want the latest in time sequence. This is a typical time sequence analytical use case. A good method for this problem is to merge both tables in a common time sequence.

#### 4.4.1.1 Define A Common Schema Holding “quotes” and “temp_trade_moving_avg” Records
This is a necessary step before the union of two datasets which have a different schema (denormalization). The schema needs to include all the fields of quotes and temp_trade_mov_avg so that no information gets lost.<

In [0]:
""" 
Schema:
Column         |  Value
--------------------------------------------------
trade_dt       |  Value from corresponding records
rec_type       |  “Q” for quotes, “T” for trades
stock_symbol   |  Value from corresponding records
event_tm       |  Value from corresponding records
event_seq_nb   |  From quotes, null for trades
exchange       |  Value from corresponding records
bid_pr         |  From quotes, null for trades
bid_size       |  From quotes, null for trades
ask_pr         |  From quotes, null for trades
ask_size       |  From quotes, null for trades
trade_pr       |  From trades, null for quotes
mov_avg_pr     |  From trades, null for quotes
"""

Out[61]: ' Schema:\nColumn | Value\n--------------------------------------------------\ntrade_dt | Value from corresponding records\nrec_type | “Q” for quotes, “T” for trades\nsymbol | Value from corresponding records\nevent_tm | Value from corresponding records\nevent_seq_nb | From quotes, null for trades\nexchange | Value from corresponding records\nbid_pr | From quotes, null for trades\nbid_size | From quotes, null for trades\nask_pr | From quotes, null for trades\nask_size | From quotes, null for trades\ntrade_pr | From trades, null for quotes\nmov_avg_pr | From trades, null for quotes\n'

In [0]:
common_event_schema = StructType([StructField('trade_dt', DateType(), True),
                           StructField('rec_type', StringType(), True),
                           StructField('stock_symbol', StringType(), True),
                           StructField('stock_exchange', StringType(), True),
                           StructField('event_tm', TimestampType(), True),
                           StructField('event_seq_nb', IntegerType(), True),
                           StructField('bid_pr', FloatType(), True),
                           StructField('bid_size', IntegerType(), True),
                           StructField('ask_pr', FloatType(), True),
                           StructField('ask_size', IntegerType(), True),
                           StructField('trade_pr', FloatType(), True),
                           StructField('mov_avg_pr', FloatType(), True)])

#### 4.4.1.2 Create Spark Temp View To Union Both Tables
Perform data normalization in the select statement from both table followed by union:

In [0]:
"""PROPOSED TEMPLATE QUOTE:

quote_union = spark.sql("
# [Creates a denormalized view union both quotes and
temp_trade_moving_avg, populate null for fields not applicable]
")
quote_union.createOrReplaceTempView("quote_union")
"""


Out[191]: 'PROPOSED TEMPLATE QUOTE:\n\nquote_union = spark.sql("\n# [Creates a denormalized view union both quotes and\ntemp_trade_moving_avg, populate null for fields not applicable]\n")\nquote_union.createOrReplaceTempView("quote_union")\n'

In [0]:
df =spark.read.parquet("/quote/trade_dt={}".format("2020-08-06"))
df.createOrReplaceTempView("quotes")

In [0]:
quote_union = spark.sql("SELECT date(latest_trade) as trade_dt, 'T' as rec_type,stock_symbol,stock_exchange,latest_trade as event_tm, event_seq_nb, null as bid_pr,null as bid_size,null as ask_pr, null as ask_size, trade_pr,mov_avg_pr from temp_trade_moving_avg").union(spark.sql("SELECT trade_dt, 'Q' as rec_type,stock_symbol,stock_exchange,latest_quote as event_tm, event_seq_nb, bid_pr,bid_size,ask_pr, ask_size, null as trade_pr,null as mov_avg_pr from quotes"))
quote_union.createOrReplaceTempView("quote_union")

In [0]:
yesterday_very_last_trades = spark.sql("SELECT date(latest_trade) AS trade_dt, 'T' AS rec_type,stock_symbol, stock_exchange, latest_trade as event_tm, event_seq_nb,null as bid_pr,null as bid_size, null as ask_pr,null as ask_size,last_trade_pr AS trade_pr, last_mov_avg_pr AS mov_avg_pr FROM (SELECT stock_symbol, stock_exchange, latest_trade,  event_seq_nb, LAST_VALUE(trade_pr) OVER (PARTITION BY stock_symbol,stock_exchange ORDER BY latest_trade RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS last_trade_pr, LAST_VALUE(last_mov_avg_pr) OVER (PARTITION BY stock_symbol,stock_exchange ORDER BY latest_trade RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS last_mov_avg_pr FROM prev_temp_last_trade) AS prev_last_trades")

w = Window.partitionBy('stock_symbol','stock_exchange').orderBy(desc('event_tm'))
ydf = yesterday_very_last_trades.withColumn('Rank',dense_rank().over(w))
yesterday_very_last_trades = ydf.filter(ydf.Rank == 1).drop(ydf.Rank).orderBy('event_tm')


In [0]:
print("These are the very last trade prices and moving average trade prices of the previous day, per symbol and stock exchange: ")
yesterday_very_last_trades.show(truncate=False)

These are the very last trade prices and moving average trade prices of the previous day, per symbol and stock exchange: 
+----------+--------+------------+--------------+-----------------------+------------+------+--------+------+--------+---------+-----------------+
trade_dt |rec_type|stock_symbol|stock_exchange|event_tm |event_seq_nb|bid_pr|bid_size|ask_pr|ask_size|trade_pr |mov_avg_pr |
+----------+--------+------------+--------------+-----------------------+------------+------+--------+------+--------+---------+-----------------+
2020-08-05|T |SYMA |NYSE |2020-08-05 14:24:34.33 |40 |null |null |null |null |78.430954|76.67514546712239|
2020-08-05|T |SYMC |NASDAQ |2020-08-05 17:02:58.71 |60 |null |null |null |null |160.4474 |158.7165781656901|
2020-08-05|T |SYMB |NYSE |2020-08-05 21:27:28.314|100 |null |null |null |null |33.956287|34.15988794962565|
2020-08-05|T |SYMA |NASDAQ |2020-08-05 21:40:23.197|100 |null |null |null |null |77.24676 |77.03174845377605|
2020-08-05|T |SYMC |NYSE |2020-08-05 21:52:38.607|100 |null |null |null |null |160.61949|160.4873504638672|
+----------+--------+------------+--------------+-----------------------+------------+------+--------+------+--------+---------+-----------------+

In [0]:
quote_union_all = yesterday_very_last_trades.union(spark.sql("SELECT * FROM quote_union")).orderBy('stock_exchange','stock_symbol','event_tm','rec_type')

In [0]:
print('Now including previous days last state: ')
quote_union_all.show(truncate=False)

Now including previous days last state: 
+----------+--------+------------+--------------+-----------------------+------------+---------+--------+---------+--------+--------+-----------------+
trade_dt |rec_type|stock_symbol|stock_exchange|event_tm |event_seq_nb|bid_pr |bid_size|ask_pr |ask_size|trade_pr|mov_avg_pr |
+----------+--------+------------+--------------+-----------------------+------------+---------+--------+---------+--------+--------+-----------------+
2020-08-05|T |SYMA |NASDAQ |2020-08-05 21:40:23.197|100 |null |null |null |null |77.24676|77.03174845377605|
2020-08-06|Q |SYMA |NASDAQ |2020-08-06 11:32:52.994|16 |76.65085 |100 |77.56526 |100 |null |null |
2020-08-06|Q |SYMA |NASDAQ |2020-08-06 11:40:09.89 |17 |75.345146|100 |76.79113 |100 |null |null |
2020-08-06|Q |SYMA |NASDAQ |2020-08-06 11:49:18.978|18 |75.37165 |100 |76.572685|100 |null |null |
2020-08-06|Q |SYMA |NASDAQ |2020-08-06 11:55:10.734|19 |75.16338 |100 |75.755356|100 |null |null |
2020-08-06|T |SYMA |NASDAQ |2020-08-06 12:00:29.595|20 |null |null |null |null |77.0967 |75.79523849487305|
2020-08-06|Q |SYMA |NASDAQ |2020-08-06 13:33:21.066|33 |75.983345|100 |77.97206 |100 |null |null |
2020-08-06|Q |SYMA |NASDAQ |2020-08-06 13:39:12.101|34 |79.13626 |100 |79.67412 |100 |null |null |
2020-08-06|Q |SYMA |NASDAQ |2020-08-06 13:47:53.534|35 |76.2566 |100 |76.693275|100 |null |null |
2020-08-06|Q |SYMA |NASDAQ |2020-08-06 13:55:29.699|36 |78.07561 |100 |78.98503 |100 |null |null |
2020-08-06|Q |SYMA |NASDAQ |2020-08-06 14:10:18.336|38 |75.48581 |100 |75.982895|100 |null |null |
2020-08-06|Q |SYMA |NASDAQ |2020-08-06 14:35:11.172|41 |75.970894|100 |77.6246 |100 |null |null |
2020-08-06|Q |SYMA |NASDAQ |2020-08-06 14:49:49.235|43 |77.786896|100 |77.89292 |100 |null |null |
2020-08-06|Q |SYMA |NASDAQ |2020-08-06 15:21:01.193|47 |75.40022 |100 |76.63095 |100 |null |null |
2020-08-06|Q |SYMA |NASDAQ |2020-08-06 15:26:11.753|48 |76.66278 |100 |78.51535 |100 |null |null |
2020-08-06|Q |SYMB |NASDAQ |2020-08-06 09:39:12.645|1 |31.902136|100 |32.71381 |100 |null |null |
2020-08-06|Q |SYMB |NASDAQ |2020-08-06 09:46:22.396|2 |32.90886 |100 |33.66323 |100 |null |null |
2020-08-06|Q |SYMB |NASDAQ |2020-08-06 09:59:49.083|4 |32.63884 |100 |32.841022|100 |null |null |
2020-08-06|Q |SYMB |NASDAQ |2020-08-06 10:15:27.803|6 |34.879185|100 |36.021904|100 |null |null |
2020-08-06|Q |SYMB |NASDAQ |2020-08-06 10:23:05.541|7 |31.81192 |100 |32.962135|100 |null |null |
+----------+--------+------------+--------------+-----------------------+------------+---------+--------+---------+--------+--------+-----------------+
only showing top 20 rows

#### 4.4.1.3 Populate The Latest trade_pr and mov_avg_pr
Use a window analytical function to populate the values from the latest records with rec_type “T.” The window should be under the partition of the exchange symbol:

In [0]:
"""
quote_union_update = spark.sql("""
select *,
# [logic for the last not null trade price] AS last_trade_pr,
# [logic for the last not null mov_avg_pr price] AS last_mov_avg_pr
from quote_union
""")
quote_union_update.createOrReplaceTempView("quote_union_update")
"""

#### 4.4.1.4 Filter For Quote Records
Since you ultimately only need quote records, filter out trade events after the calculation.

In [0]:
quote_update = spark.sql("""
select trade_dt, symbol, event_tm, event_seq_nb, exchange,
bid_pr, bid_size, ask_pr, ask_size, last_trade_pr, last_mov_avg_pr
from quote_union_update
where rec_type = 'Q'
""")
quote_update.createOrReplaceTempView("quote_update")

### 4.4.2 Join With Table temp_last_trade To Get The Prior Day Close Price
The prior day close price table has a single record per symbol and exchange. For this join, you can use equality join since the join fields are only “symbol” and “exchange”. However, note that this table has a very limited number of records (no more than the number of symbol and exchange combinations). This is an excellent opportunity to use broadcast join to achieve optimal join performance. Broadcast join is always recommended if one of the join tables is small enough so that the whole dataset fits in memory.

In [0]:
quote_final = spark.sql("""
select trade_dt, symbol, event_tm, event_seq_nb, exchange,
bid_pr, bid_size, ask_pr, ask_size, last_trade_pr, last_mov_avg_pr,
bid_pr - close_pr as bid_pr_mv, ask_pr - close_pr as ask_pr_mv
from (
# [Broadcast temp_last_trade table. Use quote_update to left outer join
temp_last_trade]
) a
""")

### 4.4.3 Write The Final Dataframe Into Azure Blob Storage At Corresponding Partition'
The final output needs to be stored as a new partition for the object on Azure Blob Storage.

In [0]:
quote_update.write.parquet("cloud-storage-path/quote-trade-analytical/date={}".format(trade_date))